In [29]:

# =========================
# [1] IMPORTS Y CONSTANTES
# =========================
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
import serial
import serial.tools.list_ports
import threading
import time
import json
import pickle
from math import inf
import math
import tkintermapview
from geographiclib.geodesic import Geodesic
import uuid
from PIL import Image, ImageTk

# Protocolo de comunicación
HEADER_OUT = "GS#"    # lo que ENVÍA la base
HEADER_IN  = "UAV#"   # lo que RECIBE la base
FOOTER     = "#END"

# Serial
SERIAL_PORT = None
SERIAL_BAUDRATE = 115200
ser = None
is_connected = False

# ACK
ACK_TIMEOUT = 1.5        # segundos para esperar ACK
MAX_RETRIES = 3          # reintentos máximos
pending_acks = {}        # { id: {"frame": str, "timestamp": float, "retries": int} }
ack_lock = threading.Lock()

# Estado
last_telemetry_pos = None

# Edición de mapa
EDIT_MODE = 'polygon'  # 'move', 'delete', 'home', 'polygon'

# =========================
# [2] ESTRUCTURAS DE DATOS
# =========================
class MissionData:
    def __init__(self):
        self.polygon_points = []   # [(lat, lon), ...]
        self.home_position = None  # (lat, lon) o None
        self.altitude = 3         # Altitud de misión (m)
        self.spacing = 5          # Ancho entre pasadas (m)
        self.detect_spacing = 5 

    def to_dict(self):
        """Convierte a diccionario compacto para LoRa"""
        return {
            "p": [[round(lat, 6), round(lon, 6)] for lat, lon in self.polygon_points],
            "h": [round(self.home_position[0], 6), round(self.home_position[1], 6)] if self.home_position else None,
            "a": self.altitude,
            "s": self.spacing,
            "ds": self.detect_spacing,
        }

    def from_dict(self, data):
        """Carga desde diccionario"""
        self.polygon_points = [tuple(point) for point in data.get("p", [])]
        self.home_position = tuple(data["h"]) if data.get("h") else None
        self.altitude = data.get("a", 3)
        self.spacing = data.get("s", 5)
        self.detect_spacing = data.get("ds", 5)

# =========================
# [3] VARIABLES GLOBALES GUI
# =========================
# Mapa y marcadores
polygon_markers = []
polygon_path = None
home_marker = None
home_line = None
mission_path = None
waypoint_markers = []
waypoint_status = []
fire_markers = []
person_markers = []
current_target_marker = None
mission_tracking_enabled = False
battery_fail_marker = None
battery_low_alert_shown = False
esp_wp_chunks = []
esp_wp_total = 0
esp_mission_path = None
esp_wp_markers = []
mission_wp_markers = []
mission_ready_flag = False
sent_chunks_cache = []

# Controles / widgets
btn_disarm = None
btn_mission = None
btn_return = None
btn_arm = None
start_mission_btn = None

event_action_var = None
port_var = None
spacing_var = None
altitude_var = None
detect_spacing_var = None   
status_message = None
home_position_var = None
progress_var = None
progress_bar = None
battery_low_alert_shown = False
mission_sent = False
home_set = False
polygon_set = False
drone_armed = False


# Gripper
gripper_state_var = None

# Vistas
root = None
mapa = None
telemetry_text = None
log_text = None

#iconos
active_wp_marker = None      # marcador del waypoint actual
waypoint_list = []           # coordenadas de cada waypoint enviadas por el dron
waypoint_total = 0

flight_track = []     # lista de puntos reales recorridos
flight_path = None    # path verde en el mapa

def safe_delete(obj):
    try:
        root.after_idle(lambda: obj.delete())
    except:
        pass

# =========================
# [4] UTILIDADES GEODÉSICAS
# =========================
def calculate_distance(lat1, lon1, lat2, lon2):
    """Distancia en metros entre dos puntos"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['s12']

def calculate_bearing(lat1, lon1, lat2, lon2):
    """Rumbo en grados entre dos puntos"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['azi1']

def get_point_at_distance(lat1, lon1, bearing, distance):
    """Punto desde (lat1, lon1) a distancia (m) y rumbo (°)"""
    result = Geodesic.WGS84.Direct(lat1, lon1, bearing, distance)
    return (result['lat2'], result['lon2'])

# =============================================
# [4B] GENERACIÓN DE PATRÓN LAWNMOWER EN LA GS
# =============================================

def rotate_point(lat, lon, origin_lat, origin_lon, angle_deg):
    """
    Convierte (lat, lon) a coordenadas locales en METROS respecto al origen
    y las rota angle_deg grados. Devuelve (x_rot, y_rot) en metros.
    """
    # 1) Pasar de lat/lon a sistema local (x,y) en metros
    dx = calculate_distance(origin_lat, origin_lon, origin_lat, lon)
    if lon < origin_lon:
        dx = -dx

    dy = calculate_distance(origin_lat, origin_lon, lat, origin_lon)
    if lat < origin_lat:
        dy = -dy

    # 2) Rotar en el plano local
    angle = math.radians(angle_deg)
    xr = dx * math.cos(angle) - dy * math.sin(angle)
    yr = dx * math.sin(angle) + dy * math.cos(angle)

    # 3) OJO: acá NO volvemos a lat/lon.
    #     Devolvemos coordenadas locales rotadas en METROS.
    return xr, yr


def rotate_polygon(points, origin, angle_deg):
    return [rotate_point(p[0], p[1], origin[0], origin[1], angle_deg) for p in points]

def local_to_latlon(x, y, origin_lat, origin_lon, angle_deg):
    """
    Convierte coordenadas locales (x,y) EN METROS, rotadas, 
    nuevamente a coordenadas geográficas (lat/lon).
    angle_deg = ángulo original del polígono respecto al norte.
    """
    # Rotar de vuelta al sistema global
    angle = math.radians(angle_deg)
    xr = x * math.cos(angle) - y * math.sin(angle)
    yr = x * math.sin(angle) + y * math.cos(angle)

    # Convertir valores locales (xr, yr) de metros a lat/lon reales
    lat = get_point_at_distance(origin_lat, origin_lon, 0, yr)[0]
    lon = get_point_at_distance(origin_lat, origin_lon, 90, xr)[1]

    return lat, lon


def generate_lawnmower_path(polygon, spacing, ds, home=None):
    """
    Genera el patrón lawnmower COMPLETO para mostrar en GS.
    polygon: lista [(lat, lon), ...]
    spacing: separación entre pasadas (en metros)
    ds: distancia entre puntos de detección (en metros)
    """
    if len(polygon) < 3:
        return []

    # -------------------------
    # 1. Encontrar lado mayor
    # -------------------------
    max_len = -1
    pA, pB = polygon[0], polygon[1]
    for i in range(len(polygon)):
        for j in range(i+1, len(polygon)):
            d = calculate_distance(polygon[i][0], polygon[i][1],
                                   polygon[j][0], polygon[j][1])
            if d > max_len:
                max_len = d
                pA, pB = polygon[i], polygon[j]

    # rumbo del lado mayor
    bearing = calculate_bearing(pA[0], pA[1], pB[0], pB[1])

    # -------------------------
    # 2. Rotar polígono para que lado mayor quede horizontal
    # -------------------------
    origin = pA
    polygon_rot = rotate_polygon(polygon, origin, -bearing)

    # -------------------------
    # 3. Encontrar Y min y Y max
    # -------------------------
    ys = [pt[1] for pt in polygon_rot]
    min_y, max_y = min(ys), max(ys)

    # -------------------------
    # 4. Generar pasadas a intervalos "spacing"
    # -------------------------
    path = []
    y = min_y
    direction = 1

    while y <= max_y:
        # Intersecciones de borde con esta línea horizontal "y"
        intersections = []
        for i in range(len(polygon_rot)):
            x1, y1 = polygon_rot[i]
            x2, y2 = polygon_rot[(i + 1) % len(polygon_rot)]

            # chequear si cruza
            if (y1 <= y <= y2) or (y2 <= y <= y1):
                if y1 == y2:
                    continue  # borde horizontal, ignorar
                t = (y - y1) / (y2 - y1)
                x = x1 + t * (x2 - x1)
                intersections.append(x)

        intersections.sort()

        if len(intersections) >= 2:
            x_start, x_end = intersections[0], intersections[-1]

            # ----------------------------
            # Crear puntos cada "ds" metros
            # ----------------------------
            segment_length = abs(x_end - x_start)
            n_points = max(2, int(segment_length / ds))

            if direction == 1:
                xs = [x_start + (segment_length * i / (n_points - 1))
                      for i in range(n_points)]
            else:
                xs = [x_end - (segment_length * i / (n_points - 1))
                      for i in range(n_points)]

            # ----------------------------
            # Convertir x,y rotado → lat/lon real
            # ----------------------------
            for x in xs:
                lat, lon = local_to_latlon(x, y, origin[0], origin[1], bearing)
                path.append((lat, lon))

        direction *= -1
        y += spacing
    

    return path


def draw_esp_mission_path(coords):
    """
    Dibuja el trazado enviado por el ESP.
    Se fuerza visualización con:
    - Marcadores en TODOS los puntos
    - Auto-zoom al centro
    - Path celeste robusto
    """

    global esp_mission_path, esp_wp_markers

    # ---------------------------------------------------------
    # BORRAR ANTERIORES
    # ---------------------------------------------------------
    if esp_mission_path:
        try: safe_delete(esp_mission_path)
        except: pass
        esp_mission_path = None

    for m in esp_wp_markers:
        try: safe_delete(m)
        except: pass
    esp_wp_markers.clear()

    # ---------------------------------------------------------
    # VALIDACIÓN
    # ---------------------------------------------------------
    if not coords:
        log_message("⚠️ draw_esp_mission_path: coords vacío")
        return

    log_message(f"📌 {len(coords)} waypoints recibidos del ESP")

    # ---------------------------------------------------------
    # LOGUEAR TODOS LOS PUNTOS (debug)
    # ---------------------------------------------------------
    for i, (lat, lon) in enumerate(coords):
        log_message(f"   [{i}] {lat:.7f}, {lon:.7f}")

    # ---------------------------------------------------------
    # AUTO-ZOOM
    # ---------------------------------------------------------
    try:
        lats = [c[0] for c in coords]
        lons = [c[1] for c in coords]
        mid_lat = (min(lats) + max(lats)) / 2
        mid_lon = (min(lons) + max(lons)) / 2

        map_widget.set_position(mid_lat, mid_lon)
        map_widget.set_zoom(19)
        log_message("🔍 Auto-zoom centrado en ruta ESP")
    except Exception as e:
        log_message(f"⚠️ Error auto-zoom: {e}")

    # ---------------------------------------------------------
    # MARCADORES: TODOS LOS PUNTOS
    # ---------------------------------------------------------
    for i, (lat, lon) in enumerate(coords):
        try:
            marker = map_widget.set_marker(lat, lon, text=f"{i}")
            esp_wp_markers.append(marker)
        except Exception as e:
            log_message(f"⚠️ Error dibujando marker {i}: {e}")

    # ---------------------------------------------------------
    # PATH CELESTE
    # ---------------------------------------------------------
    try:
        # Forzamos formato correcto
        path_coords = [(float(lat), float(lon)) for lat, lon in coords]

        esp_mission_path = map_widget.set_path(
            path_coords,
            color="#00CFFF",
            width=3
        )
        log_message("🧭 Path ESP dibujado correctamente (celeste)")
    except Exception as e:
        log_message(f"❌ Error dibujando path ESP: {e}")



def update_buttons_state():
    # Seguridad: si los botones aún no están creados, no hacer nada
    if 'send_mission_btn' not in globals() or send_mission_btn is None:
        return
    if 'start_mission_btn' not in globals() or start_mission_btn is None:
        return

    # --- botón ENVIAR MISIÓN ---
    if home_set and polygon_set:
        send_mission_btn.config(state="normal")
    else:
        send_mission_btn.config(state="disabled")

    # --- botón COMENZAR MISIÓN ---
    if drone_armed and mission_ready_flag:
        start_mission_btn.config(state="normal")
    else:
        start_mission_btn.config(state="disabled")

# =========================================================
# [5] LOG Y TELEMETRÍA (SEGUROS PARA LLAMAR DESDE HILOS)
# =========================================================
def send_ack_to_esp(msg_id):
    if not msg_id:
        return
    message = {
        "t": "ACK",
        "id": msg_id,
        "ts": int(time.time() * 1000)
    }
    payload = json.dumps(message, separators=(",", ":"))
    frame = f"{HEADER_OUT}{payload}{FOOTER}"
    ser.write(frame.encode())


def log_message(message):
    """Agrega texto al log de manera segura desde cualquier hilo."""
    try:
        timestamp = time.strftime('%H:%M:%S')
        full_msg = f"{timestamp} - {message}\n"

        if 'log_text' not in globals() or not log_text:
            print(full_msg.strip())  # fallback si GUI aún no existe
            return

        def update_log():
            log_text.config(state="normal")
            log_text.insert("end", full_msg)
            log_text.see("end")
            log_text.config(state="disabled")

        root.after(0, update_log)
    except Exception as e:
        print(f"[Log Error] {e} :: {message}")

def update_telemetry(data):
    if not root:
        return

    def update_gui():
        global last_telemetry_pos, battery_low_alert_shown, battery_fail_marker

        telemetry_text.config(state="normal")
        telemetry_text.delete(1.0, "end")

        voltage = data.get('voltage', 'N/A')
        battery_pct = data.get('battery', 'N/A')
        batt_warn = data.get('batt_warn', False)

        telemetry_str = f"Latitud: {data.get('lat', 'N/A')}\n"
        telemetry_str += f"Longitud: {data.get('lon', 'N/A')}\n"
        telemetry_str += f"Altitud: {data.get('alt', 'N/A')} m\n"
        telemetry_str += f"Velocidad: {data.get('speed', 'N/A')} km/h\n"
        telemetry_str += f"Rumbo: {data.get('heading', 'N/A')}°\n"
        telemetry_str += f"Batería: {battery_pct}%\n"
        telemetry_str += f"Voltaje: {voltage} V\n"
        telemetry_str += f"Estado: {data.get('status', 'N/A')}\n"
        telemetry_str += f"Waypoint actual: {data.get('current_wp', 'N/A')}\n"
        telemetry_str += f"Waypoints totales: {data.get('total_wp', 'N/A')}\n"

        telemetry_text.insert("end", telemetry_str)
        telemetry_text.config(state="disabled")

        # ========= 🔥 EVENTO: BATERIA BAJA =========
        if batt_warn and not battery_low_alert_shown:

            battery_low_alert_shown = True

            # Popup visual
            if voltage != "N/A":
                show_battery_popup(float(voltage))

            # Colocar marcador en mapa
            if 'lat' in data and 'lon' in data and data['lat'] != 'N/A' and data['lon'] != 'N/A':
                place_battery_fail_marker(data['lat'], data['lon'])
                log_message("📍 Marcador de batería baja colocado en el mapa")

        # ==========================================

        # Marcador del dron y ruta
        if 'lat' in data and 'lon' in data and data['lat'] != 'N/A' and data['lon'] != 'N/A':
            lat = data['lat']
            lon = data['lon']
            last_telemetry_pos = (lat, lon)
            update_drone_position(lat, lon)
            update_flight_track(lat, lon)

    root.after(0, update_gui)



def show_battery_popup(voltage):
    message = f"⚠️ BATERÍA BAJA\n\nVoltaje: {voltage:.2f} V\nEl dron está en riesgo."
    messagebox.showerror("⚠️ BATERÍA BAJA", message)

def place_battery_fail_marker(lat, lon):
    global battery_fail_marker

    if map_widget is None:
        return

    battery_fail_marker = map_widget.set_marker(
        lat, lon,
        text="Batería baja",
        marker_color_circle="red",
        marker_color_outside="darkred"
    )


def update_drone_position(lat, lon):
    """Dibuja/actualiza el marcador del dron en el mapa."""
    if hasattr(update_drone_position, "marker") and update_drone_position.marker:
        update_drone_position.marker.delete()

    update_drone_position.marker = mapa.set_marker(
        lat, lon, text="DRON",
        marker_color_circle="red", marker_color_outside="red",
        text_color="red", font=("Helvetica", 12, "bold")
    )

    current_center = mapa.get_position()
    if calculate_distance(current_center[0], current_center[1], lat, lon) > 5000:
        mapa.set_position(lat, lon)
        mapa.set_zoom(15)
        
def update_flight_track(lat, lon):
    global flight_track, flight_path, mission_tracking_enabled

    # 🚫 No trazar si la misión no está activa
    if not mission_tracking_enabled:
        return

    # 🚫 Anti-jitter: no trazar puntos que no representen desplazamiento real
    if flight_track:
        last_lat, last_lon = flight_track[-1]
        d = calculate_distance(last_lat, last_lon, lat, lon)
        if d < 0.8:     # menos de 80 cm → jitter
            return

    # Agregar punto válido
    flight_track.append((lat, lon))

    # Borrar visualización previa
    if flight_path:
        try:
             safe_delete(flight_path)
        except:
            pass

    # Dibujar trayecto actualizado
    if len(flight_track) > 1:
        flight_path = mapa.set_path(flight_track, color="red", width=3)


def create_active_wp_marker(lat, lon, wp_id):
    return mapa.set_marker(
        lat, lon,
        text=f"WP{wp_id}",
        marker_color_circle="#FFD000",   # amarillo fuerte
        marker_color_outside="#A67C00",  # borde marrón oscuro
        text_color="black",
        font=("Helvetica", 11, "bold")
    )
def update_active_waypoint(lat, lon, wp_id):
    global active_wp_marker

    # borrar anterior
    if active_wp_marker:
        safe_delete(active_wp_marker)

    # crear el nuevo
    active_wp_marker = create_active_wp_marker(lat, lon, wp_id)

def handle_wp_chunk(data):
    """
    Reconstruye los fragmentos de waypoints enviados por el ESP (WP_CHUNK)
    y, cuando están todos, dibuja el trazado en celeste en el mapa.
    """
    global esp_wp_chunks, esp_wp_total

    idx = data.get("i")
    total = data.get("n")
    chunk = data.get("chunk", "")

    if idx is None or total is None:
        log_message(f"⚠️ WP_CHUNK inválido: {data}")
        return

    # Inicializar buffer SOLO si cambió el total o si está vacío
    if esp_wp_total != total or len(esp_wp_chunks) != total:
        esp_wp_total = total
        esp_wp_chunks = [""] * total
        log_message(f"🧩 Iniciando recepción de {total} fragmentos de waypoints")

    # Insertar chunk
    if 0 <= idx < esp_wp_total:
        esp_wp_chunks[idx] = chunk
    else:
        log_message(f"⚠️ Índice fuera de rango: i={idx}, total={esp_wp_total}")
        return

    # Si ya están todos los fragmentos → reconstruir cadena completa
    if all(esp_wp_chunks):
        big_str = "".join(esp_wp_chunks)

        # Parsear: "lat,lon;lat,lon;..."
        pairs = [p for p in big_str.strip().split(";") if p.strip()]
        coords = []

        for p in pairs:
            try:
                lat_s, lon_s = p.split(",")
                coords.append((float(lat_s), float(lon_s)))
            except Exception as e:
                log_message(f"⚠️ Error parseando par '{p}': {e}")

        log_message(f"🧭 {len(coords)} waypoints reconstruidos desde ESP")

        draw_esp_mission_path(coords)



# =====================================
# [6] COMUNICACIÓN: CONEXIÓN / ENVÍOS
# =====================================
def connect_serial():
    """Conecta el puerto serial y lanza los hilos de recepción y ACK."""
    global ser, is_connected, SERIAL_PORT

    is_connected = False
    SERIAL_PORT = port_var.get()

    if not SERIAL_PORT:
        messagebox.showerror("Error", "Selecciona un puerto serial primero")
        return False

    try:
        ser = serial.Serial(SERIAL_PORT, SERIAL_BAUDRATE, timeout=1)
        ser.reset_input_buffer()
        ser.reset_output_buffer()

        is_connected = True
        print(f"[SERIAL] ✅ Conectado a {SERIAL_PORT} @ {SERIAL_BAUDRATE} bps")
        status_label.config(text=f"Conectado: {SERIAL_PORT}", fg="green")
        log_message(f"✅ Conectado correctamente a {SERIAL_PORT} a {SERIAL_BAUDRATE} bps")

        if 'root' in globals() and root:
            print("[SERIAL] 🚀 Lanzando hilos: recepción y ack_manager")
            threading.Thread(target=receive_data, daemon=True).start()
            threading.Thread(target=ack_manager, daemon=True).start()
        else:
            print("[SERIAL] ⚠️ GUI no inicializada aún, no se lanzan hilos")

        return True

    except Exception as e:
        is_connected = False
        print(f"[SERIAL] ❌ Error conectando a {SERIAL_PORT}: {e}")
        log_message(f"❌ Error conectando al puerto {SERIAL_PORT}: {e}")
        messagebox.showerror("Error", f"No se pudo conectar: {str(e)}")
        return False

def disconnect_serial():
    """Desconecta el puerto serial de forma segura y limpia."""
    global ser, is_connected

    if not is_connected:
        print("[SERIAL] ⚠️ No hay conexión activa para cerrar.")
        log_message("⚠️ No hay conexión activa para cerrar.")
        return

    try:
        is_connected = False
        time.sleep(0.2)  # permitir que terminen hilos
        if ser and ser.is_open:
            ser.close()
        status_label.config(text="Desconectado", fg="red")
        log_message("🔌 Puerto serial cerrado correctamente")
        print("[SERIAL] ✅ Puerto cerrado correctamente")
    except Exception as e:
        log_message(f"⚠️ Error al cerrar el puerto serial: {e}")
        print(f"[SERIAL] ⚠️ Error al cerrar puerto: {e}")
    finally:
        ser = None

def send_command(command_type, data=None, require_ack=True):
    """Forma el frame GS#...#END, envía y registra para ACK si corresponde."""
    if not is_connected:
        messagebox.showwarning("Error", "No hay conexión serial")
        return

    # ID único del paquete
    packet_id = str(uuid.uuid4())[:8]

    message = {
        "t": command_type,
        "d": data if data else {},
        "ts": int(time.time() * 1000),
        "id": packet_id
    }

    payload = json.dumps(message, separators=(",", ":"))
    frame   = f"{HEADER_OUT}{payload}{FOOTER}"

    try:
        # 🟦 Print de envío completo
        print(f"[TX → LoRa/send_command] Frame enviado → {frame}")
        
        ser.write(frame.encode())
        log_message(f"Enviado: {frame}")

        if require_ack and command_type != "TELEMETRY":
            with ack_lock:
                pending_acks[packet_id] = {
                    "frame": frame,
                    "timestamp": time.time(),
                    "retries": 0
                }

    except Exception as e:
        log_message(f"Error enviando: {str(e)}")

def request_esp_waypoints():
    send_command("REQ_WP_DEBUG", {})
    log_message("📡 Solicitado trazado del dron…")


# ========================================
# [7] COMUNICACIÓN: RECEPCIÓN Y ACKs
# ========================================
def ack_manager():
    """Reintenta envíos sin ACK dentro del timeout."""
    while is_connected:
        time.sleep(0.2)
        now = time.time()

        with ack_lock:
            to_delete = []
            for pid, info in list(pending_acks.items()):
                if now - info["timestamp"] > ACK_TIMEOUT:
                    if info["retries"] < MAX_RETRIES:
                        try:
                            ser.write(info["frame"].encode())
                            pending_acks[pid]["timestamp"] = now
                            pending_acks[pid]["retries"] += 1
                            print(f"[ACK MANAGER → LoRa] 🔄 Reintentando ID={pid} (reintento {info['retries']})")
                            log_message(f"🔄 Reintentando envío ID={pid}")
                        except Exception as e:
                            print(f"[ACK MANAGER] ⚠️ Error reintentando ID={pid}: {e}")
                            log_message(f"Error reintentando ID={pid}: {e}")
                    else:
                        log_message(f"❌ No se recibió ACK para ID={pid}")
                        print(f"[ACK MANAGER] ❌ ACK no recibido tras {MAX_RETRIES} intentos → ID={pid}")
                        to_delete.append(pid)

            for pid in to_delete:
                del pending_acks[pid]

def receive_data():
    """Lee y procesa frames UAV#...#END desde el serial."""
    global is_connected
    buffer = ""

    while is_connected:
        if not ser or not ser.is_open:
            break
        try:
            if ser.in_waiting > 0:
                chunk = ser.read(ser.in_waiting).decode(errors="ignore")
                buffer += chunk

                while True:
                    h = buffer.find(HEADER_IN)
                    if h < 0:
                        if len(buffer) > 4096:
                            buffer = buffer[-512:]
                        break

                    f = buffer.find(FOOTER, h)
                    if f < 0:
                        if h > 0: buffer = buffer[h:]
                        break

                    payload = buffer[h + len(HEADER_IN):f].strip()
                    buffer = buffer[f + len(FOOTER):]

                    # 🟩 Print de recepción de paquete completo
                    print(f"[RX ← LoRa/receive_data] Frame recibido ← UAV#{payload}#END")

                    try:
                        msg = json.loads(payload)
                        process_received_message(msg)
                    except Exception as e:
                        log_message(f"❌ Mensaje inválido: {payload[:80]} ({e})")

            time.sleep(0.01)
        except Exception as e:
            log_message(f"Error recibiendo datos: {e}")
            time.sleep(0.1)

    log_message("📴 Hilo de recepción terminado")

def process_received_message(message):
    """
    Procesa cualquier JSON recibido desde el dron (UAV).
    Envía ACK automáticos cuando corresponde (manteniendo framing GS#/UAV# y #END).
    """

    msg_type = message.get("t", "")
    data = message.get("d", message)   # <— CORRECTO Y BIEN INDENTADO
    packet_id = message.get("id", data.get("id", None))

    print(f"[RX ← LoRa/process_received_message] Mensaje recibido tipo '{msg_type}' → {message}")
    
    # ================================
    #  AUTO-ACK a mensajes del ESP
    # ================================
    # Si el mensaje tiene ID y no es ya un ACK, respondo con ACK
    if packet_id and msg_type != "ACK":
        send_ack_to_esp(packet_id)


    if not msg_type:
        log_message(f"⚠️ Mensaje sin tipo: {message}")
        return

    # =====================================================================================
    # ACK entrante desde UAV
    # =====================================================================================
    if msg_type == "ACK":
        print(f"[ACK IN ← LoRa/process_received_message] {message}")
        ack_id = None

        if "id" in message:
            ack_id = message["id"]
        elif isinstance(message.get("d"), dict) and "id" in message["d"]:
            ack_id = message["d"]["id"]

        if ack_id:
            with ack_lock:
                if ack_id in pending_acks:
                    del pending_acks[ack_id]
                    log_message(f"✅ ACK recibido para ID={ack_id}")
                else:
                    log_message(f"⚠️ ACK recibido con ID desconocido: {ack_id}")
        else:
            log_message("⚠️ ACK recibido sin ID válido")

        return

    # =====================================================================================
    # TELEMETRY
    # =====================================================================================
    if msg_type == "TELEMETRY":
        update_telemetry(data)
        return

    # =====================================================================================
    # STATUS
    # =====================================================================================
    elif msg_type == "STATUS":
        # Ejemplo:
        # {"t":"STATUS","state":3,"state_name":"NAVIGATE"}
        print(f"[STATUS ← LoRa] Data: {data}")
    
        # Tomamos nombre del estado
        name = (
            data.get("state_name")
            or data.get("message")
            or str(data.get("state", "DESCONOCIDO"))
        )
    
        # ================================
        # 🔥 FIX: activar/desactivar trazado
        # ================================
        global mission_tracking_enabled
    
        if name in ("NAVIGATE", "STABILIZE", "WAIT_ANALYSIS", "RETURN_HOME", "TAKEOFF"):
            mission_tracking_enabled = True
    
        elif name in ("IDLE", "COMPLETE", "LAND"):
            mission_tracking_enabled = False
            # Si querés limpiar track en el mapa:
            # clear_track_line()
    
        # =================================
    
        log_message(f"Estado FSM: {name}")
        status_message.set(f"Estado FSM: {name}")
        return
    
    # =====================================================================================
    # WAYPOINTS_INFO
    # =====================================================================================
    elif msg_type == "WAYPOINTS_INFO":
        global waypoint_total, waypoint_list
    
        total = data.get("total", 0)
    
        initialize_waypoint_tracking(total)
    
        waypoint_total = total       # ← ahora sí actualiza la global
        waypoint_list.clear()
        
        # Crear lista vacía del tamaño adecuado
        for _ in range(total):
            waypoint_list.append((None, None))   # Placeholder seguro
        
        update_active_waypoint(None, None, 0)  # marcador vacío opcional

    
        log_message(f"Total de waypoints: {total}")
        progress_bar["value"] = 0
        return

    # =====================================================================================
    # WAYPOINT_REACHED
    # =====================================================================================
    elif msg_type == "WAYPOINT_REACHED":
        wp = data.get("wp", -1)
    
        log_message(f"Waypoint alcanzado: {wp}")
        waypoint_reached(wp)  # ← ACTUALIZA barra y texto
    
        # manejo del marcador amarillo
        global active_wp_marker
    
        # último waypoint → borrar marcador
        if wp + 1 >= waypoint_total:
            if active_wp_marker:
                safe_delete(active_wp_marker)
                active_wp_marker = None
            return
    
        # si tenemos la lista (por ahora quizás solo wp0)
        if wp + 1 < len(waypoint_list):
            lat, lon = waypoint_list[wp + 1]
            update_active_waypoint(lat, lon, wp + 1)
    
        return
    
    # =====================================================================================
    # WP_UPDATE (nuevo, enviado por el dron)
    # =====================================================================================
    elif msg_type == "WP_UPDATE":
    
        wp = data.get("wp", -1)
        lat = data.get("lat")
        lon = data.get("lon")
    
        if lat is None or lon is None:
            return
    
        update_active_waypoint(lat, lon, wp)
    
        # Guardar en lista por si querés usarlo luego
        if wp < len(waypoint_list):
            waypoint_list[wp] = (lat, lon)
        else:
            waypoint_list.append((lat, lon))
    
        return

    # =====================================================================================
    # WP_CHUNK (debug de waypoints enviados por el dron para comparación)
    # =====================================================================================
    elif msg_type == "MISSION_LOADED":
        total = data.get("total_wp", 0)
        messagebox.showinfo("Misión cargada", 
                            f"El dron cargó correctamente {total} waypoints")
        mission_ready_flag = True
        # habilitar botón comenzar misión
        if start_mission_btn:
            start_mission_btn.config(state="normal")
    
        log_message(f"✅ Misión cargada en el dron ({total} WPs)")
        return
    
    
    elif msg_type == "MISSION_ERROR":
        err = data.get("error", "Error desconocido")
        messagebox.showerror("Error de misión", 
                             f"El dron reportó un error:\n{err}\n\nVolvé a enviar la misión.")
        log_message(f"❌ ERROR misión desde dron: {err}")
    
        # Deshabilitar comenzar misión si había quedado habilitado
        if start_mission_btn:
            start_mission_btn.config(state="disabled")
        return

    elif msg_type == "REQ_WP_CHUNK":
        missing = data.get("i")
        if missing is not None and 0 <= missing < len(sent_chunks_cache):
            payload = {
                "i": missing,
                "n": len(sent_chunks_cache),
                "chunk": sent_chunks_cache[missing]
            }
            send_command("MISSION_WP_CHUNK", payload)
            log_message(f"📤 Reenvío chunk solicitado: {missing}")
        return


    # =====================================================================================
    # DEBUG
    # =====================================================================================
    elif msg_type == "DEBUG":
        print(f"[DEBUG ← LoRa] Data: {data}")
        log_message(f"🪲 DEBUG: {data.get('message', '')}")
        return

    # =====================================================================================
    # FIRE
    # =====================================================================================
    elif msg_type == "FIRE":
        confidence = data.get("confidence", None)

        print(f"[FIRE ← LoRa] Data: {data}")

        if confidence is not None:
            try:
                confidence = float(confidence)
                if confidence > 1:
                    confidence /= 100.0
            except:
                confidence = None

            log_message(f"🔥 Evento FIRE detectado (confianza={confidence*100:.1f}%)")
        else:
            log_message("🔥 Evento FIRE detectado (sin dato de confianza)")

        add_fire_marker(data)
        return

    # =====================================================================================
    # PERSON
    # =====================================================================================
    elif msg_type == "PERSON":
        confidence = data.get("confidence", None)

        print(f"[PERSON ← LoRa] Data: {data}")

        if confidence is not None:
            try:
                confidence = float(confidence)
                if confidence > 1:
                    confidence /= 100.0
            except:
                confidence = None

            log_message(f"🧍 Evento PERSON detectado (confianza={confidence*100:.1f}%)")
        else:
            log_message("🧍 Evento PERSON detectado (sin dato de confianza)")

        add_person_marker(data)
        return

# ======================================
# [8] GESTIÓN DE MISIÓN (ENVÍOS, ESTADO)
# ======================================
def update_mission_parameters():
    current_mission.altitude = altitude_var.get()
    current_mission.spacing = spacing_var.get()
    current_mission.detect_spacing = detect_spacing_var.get()
    log_message("Parámetros de misión actualizados")

# Configurable: cuántos WPs por paquete
WPS_PER_CHUNK = 3

def build_gs_path():
    """
    Helper: genera el lawnmower path usando el estado de current_mission
    y actualiza el dibujo en amarillo.
    Devuelve la lista de waypoints [(lat, lon), ...]
    """
    if len(current_mission.polygon_points) < 3:
        log_message("⚠️ build_gs_path: polígono insuficiente")
        return []

    path = generate_lawnmower_path(
        current_mission.polygon_points,
        current_mission.spacing,
        detect_spacing_var.get(),
        home=current_mission.home_position
    )

    # Dibujamos en amarillo para chequear
    draw_mission_path(path)

    return path


def send_mission_chunks(path, home_lat, home_lon, altitude, event_action):
    """
    Divide el path en chunks y los envía como MISSION_INFO + MISSION_WP_CHUNK.
    """
    if not path:
        log_message("❌ send_mission_chunks: path vacío")
        return

    # ------------------------------
    # Generar lista de chunks string
    # ------------------------------
    chunks = []   # <<<<<<<< ESTA LÍNEA ES CRÍTICA

    for i in range(0, len(path), WPS_PER_CHUNK):
        subset = path[i : i + WPS_PER_CHUNK]
        s = "".join(f"{lat:.7f},{lon:.7f};" for (lat, lon) in subset)
        chunks.append(s)

    num_chunks = len(chunks)
    total_wp   = len(path)

    log_message(f"📦 Misión GS: {total_wp} WP en {num_chunks} chunks de {WPS_PER_CHUNK}")

    # -------------------------------------------------
    # Opcional: cache para reenvío si el ESP lo pide
    # -------------------------------------------------
    global sent_chunks_cache
    sent_chunks_cache = chunks.copy()   # <<<<<< YA NO FALLA

    # ------------------------------
    # 1) Enviar header MISSION_INFO
    # ------------------------------
    mission_info = {
        "home": [home_lat, home_lon],
        "alt": float(altitude),
        "spacing": float(current_mission.spacing),
        "detect_spacing": float(current_mission.detect_spacing),
        "event_action": event_action,
        "wp_total": total_wp,
        "chunks": num_chunks,
        "wps_per_chunk": WPS_PER_CHUNK,
    }

    send_command("MISSION_INFO", mission_info)
    log_message("📤 MISSION_INFO enviado al dron")

    # ------------------------------
    # 2) Enviar cada chunk numerado
    # ------------------------------
    for i, chunk_str in enumerate(chunks):
        payload = {
            "i": i,
            "n": num_chunks,
            "chunk": chunk_str,
        }
        send_command("MISSION_WP_CHUNK", payload)
        log_message(f"📤 MISSION_WP_CHUNK {i+1}/{num_chunks} enviado (len={len(chunk_str)})")


def send_compact_mission():
    """
    NUEVA VERSIÓN:
    - Verifica polígono y home
    - Actualiza parámetros de misión
    - Genera el lawnmower path en la GS
    - Envía:
        * MISSION_INFO (home, alt, spacing, etc.)
        * MISSION_WP_CHUNK (waypoints en chunks)
    - Limpia el trazado local (para luego ver el del dron si quisieras)
    """
    if len(current_mission.polygon_points) < 4:
        messagebox.showwarning("Error", "Definí al menos 4 puntos en el mapa")
        return

    if not current_mission.home_position:
        messagebox.showwarning("Error", "Establece una posición HOME primero")
        return

    # Actualizar parámetros (altitud, spacing, detect_spacing, etc.)
    update_mission_parameters()
    event_action = event_action_var.get()

    # Generar path en GS y dibujarlo en amarillo para chequear
    path = build_gs_path()
    if not path:
        messagebox.showwarning("Error", "No se pudo generar el trayecto de la misión")
        return

    # Guardamos datos básicos
    home_lat, home_lon = current_mission.home_position
    altitude = current_mission.altitude

    # (opcional) podés seguir usando mission_dict para log,
    # pero ya no lo mandamos como MISSION_COMPACT
    mission_dict = current_mission.to_dict()
    mission_dict["event_action"] = event_action
    print(f"[DEBUG] Misión dict (sin WPs): {mission_dict}")

    # -------------------------------
    # BORRAR MARCADORES Y PATH GS
    # (como hacías antes, para que luego se vea solo el del dron)
    # -------------------------------
    global mission_wp_markers, mission_path

    for m in mission_wp_markers:
        try:
            safe_delete(m)
        except:
            pass
    mission_wp_markers.clear()

    if mission_path:
        try:
            safe_delete(mission_path)
        except:
            pass
    mission_path = None

    # -------------------------------
    # Enviar misión (header + chunks)
    # -------------------------------
    send_mission_chunks(path, home_lat, home_lon, altitude, event_action)

    log_message(f"Misión enviada en chunks. Acción tras evento: {event_action}")

def prepare_and_send_mission():
    # Validaciones previas
    if len(current_mission.polygon_points) < 4:
        messagebox.showwarning("Error", "Definí al menos 4 puntos.")
        return

    if not current_mission.home_position:
        messagebox.showwarning("Error", "No hay HOME definido.")
        return

    # Actualizar parámetros
    update_mission_parameters()

    # Obtener acción tras eventos (VOLVER / CONTINUAR)
    event_action = event_action_var.get()

    # Construir path GS
    path = build_gs_path()
    if not path:
        messagebox.showerror("Error", "No se pudo generar el trazado.")
        return

    home_lat, home_lon = current_mission.home_position
    altitude = current_mission.altitude

    # Enviar misión en chunks
    send_mission_chunks(path, home_lat, home_lon, altitude, event_action)

    global mission_sent
    mission_sent = True
    update_buttons_state()

    log_message("📤 Misión enviada correctamente")

def start_mission():
    global waypoint_status, waypoint_total, mission_tracking_enabled

    # habilitar tracking en la UI
    btn_return.config(state="normal", bg="orange")
    mission_tracking_enabled = True

    # Como ya no generamos el path acá, sino en la GS y lo enviamos,
    # podemos simplemente usar el total de WPs que le mandamos al dron.
    # Si querés, podés regenerar el path local SOLO para el contador:
    path = generate_lawnmower_path(
        current_mission.polygon_points,
        current_mission.spacing,
        current_mission.detect_spacing
    )

    waypoint_total = len(path)
    waypoint_status = ["pending"] * waypoint_total
    update_waypoint_display()

    # En lugar de send_compact_mission():
    send_command("START_FLIGHT", {})
    log_message("🚁 START_FLIGHT enviado al dron")


def return_home():
   # btn_disarm.config(state="normal", bg="red")
    btn_mission.config(state="disabled", bg="lightgrey")
    #btn_return.config(state="disabled", bg="lightgrey")
    send_command("RETURN")
    print("[TX → LoRa/return_home] Enviando comando RETURN")

def arm_drone():
    if not last_telemetry_pos:
        log_message("⚠️ No hay datos de telemetría para fijar HOME")
        return
    send_command("ARM")
    log_message("🚁 Drone ARMADO")
    print("[TX → LoRa/arm_drone] Enviando comando ARM")
   # btn_disarm.config(state="disabled", bg="lightgrey")
    btn_mission.config(state="normal", bg="green", fg="white")
    btn_return.config(state="normal", bg="orange", fg="black")
    global drone_armed
    drone_armed = True
    update_buttons_state()

def disarm_drone():
    global home_marker, home_line
    if home_marker:
        safe_delete(home_marker); 
    if home_line:
        safe_delete(home_line)
    home_marker = None
    home_line = None
    current_mission.home_position = None
    home_position_var.set("")
    send_command("DISARM")
    print("[TX → LoRa/disarm_drone] Enviando comando DISARM")
    log_message("🛑 Drone DESARMADO")
    btn_mission.config(state="disabled", bg="lightgray", fg="black")
    btn_return.config(state="disabled", bg="lightgrey", fg="black")
    global drone_armed, mission_sent
    drone_armed = False
    mission_sent = False
    update_buttons_state()

# Stubs para evitar errores si se usan en el futuro
def at_home(): return True
def not_mission_started(): return True

def show_target_waypoint(lat, lon):
    global current_target_marker

    # borrar marcador anterior
    if current_target_marker is not None:
        safe_delete(current_target_marker)

    # crear nuevo marcador
    current_target_marker = mapa.set_marker(lat, lon, text="Destino")


def initialize_waypoint_tracking(total_waypoints):
    global waypoint_status
    waypoint_status = ["pending"] * total_waypoints
    update_waypoint_display()

def waypoint_reached(wp_number):
    global waypoint_status, waypoint_total

    # --- Validación básica ---
    if wp_number is None:
        log_message("⚠️ WAYPOINT_REACHED recibido sin número de waypoint")
        return

    total_wp = len(waypoint_status)

    # --- Si el dron reporta un WP fuera de rango ---
    if wp_number < 0:
        log_message(f"⚠️ WAYPOINT_REACHED inválido (<0): {wp_number}")
        return

    if wp_number >= total_wp:
        log_message(
            f"⚠️ WAYPOINT_REACHED fuera de rango: {wp_number} "
            f"(total={total_wp}). Marcando todos como completados."
        )
        # --- MEJORA OPCIONAL ---
        # Si el dron se saltea números → completar todos
        for i in range(total_wp):
            waypoint_status[i] = "completed"
        update_waypoint_display()
        return

    # --- MEJORA OPCIONAL ---
    # Si llega WP=5 pero 0..4 están sin completar → completarlos automáticamente
    for i in range(wp_number + 1):
        waypoint_status[i] = "completed"

    update_waypoint_display()
    log_message(f"Waypoint {wp_number} completado")

    # --- Último waypoint ---
    if wp_number == total_wp - 1:
        log_message("🎉 ¡Todos los waypoints completados!")


def update_waypoint_display():
    if not waypoint_status:
        return

    completed = waypoint_status.count("completed")
    total = len(waypoint_status)

    def update_gui():
        progress_var.set(f"Waypoints: {completed}/{total}")
        progress_bar['value'] = (completed / total) * 100

    root.after(0, update_gui)

# ===========================================
# [9] MAPA: EDICIÓN, DIBUJO Y MARCADORES
# ===========================================
def set_edit_mode(mode):
    global EDIT_MODE
    EDIT_MODE = mode
    modes = {
        'polygon': "Añadir Puntos de Polígono",
        'move': "Mover Marcadores",
        'delete': "Eliminar Marcadores"
    }
    status_message.set(f"Modo: {modes.get(mode, mode)}")
    log_message(f"Modo cambiado a: {modes.get(mode, mode)}")
    update_button_colors()

def update_button_colors():
    for btn in [add_btn, delete_btn]:
        btn.config(bg="SystemButtonFace")
    if EDIT_MODE == 'polygon':
        add_btn.config(bg="lightblue")
    elif EDIT_MODE == 'delete':
        delete_btn.config(bg="lightcoral")

def map_click(coords):
    lat, lon = coords
    if EDIT_MODE == 'polygon':
        current_mission.polygon_points.append((lat, lon))
        current_mission.polygon_points = order_polygon_points(current_mission.polygon_points)
        marker = mapa.set_marker(lat, lon, text=f"P{len(current_mission.polygon_points)}")
        polygon_markers.append(marker)
        update_polygon_drawing()
        refresh_mission_preview()
        log_message(f"Punto de polígono añadido: {lat:.6f}, {lon:.6f}")
    elif EDIT_MODE == 'delete':
        closest_marker = find_closest_marker(lat, lon)
        if closest_marker:
            delete_marker(closest_marker)
        else:
            log_message("No se encontró ningún marcador cercano")

def find_closest_marker(lat, lon, max_distance=1000):
    closest_marker = None
    min_distance = inf
    for i, marker in enumerate(polygon_markers):
        marker_pos = marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('polygon', i, marker)
    if home_marker:
        marker_pos = home_marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('home', 0, home_marker)
    return closest_marker

def delete_marker(marker_info):
    global home_marker, home_line
    btn_mission.config(state="disabled", bg="lightgray", fg="black")
    marker_type, index, marker = marker_info
    if marker_type == 'polygon':
        current_mission.polygon_points.pop(index)
        current_mission.polygon_points = order_polygon_points(current_mission.polygon_points)
        safe_delete(marker)
        polygon_markers.pop(index)
        for i, marker in enumerate(polygon_markers):
            marker.set_text(f"P{i + 1}")
        update_polygon_drawing()
        refresh_mission_preview()
        log_message(f"Punto P{index + 1} eliminado")
    elif marker_type == 'home':
        if home_marker:
            safe_delete(home_marker); 
        if home_line:
            safe_delete(home_line)
        home_marker = None
        home_line = None
        current_mission.home_position = None
        home_position_var.set("")
        log_message("🏠 HOME eliminado")

def set_home_position(lat, lon):
    global home_marker, home_line
    if home_marker: safe_delete(home_marker)
    if home_line: safe_delete(home_line)
    home_marker = None
    home_line = None

    current_mission.home_position = (lat, lon)
    home_marker = mapa.set_marker(lat, lon, text="HOME",
                                  marker_color_circle="white", marker_color_outside="green")
    home_position_var.set(f"{lat:.6f}, {lon:.6f}")
    log_message(f"🏠 Home position establecida: {lat:.6f}, {lon:.6f}")

    if current_mission.polygon_points:
        closest_point = find_closest_polygon_point(lat, lon)
        home_line = mapa.set_path([current_mission.home_position, closest_point], color="green", width=2)

    refresh_mission_preview()
    global polygon_set
    polygon_set = True
    update_buttons_state()


def set_home_from_telemetry():
    """Establece HOME desde última telemetría."""
    global last_telemetry_pos
    if not last_telemetry_pos:
        log_message("⚠️ No hay datos de telemetría para establecer HOME")
        messagebox.showwarning("Atención", "No hay datos de telemetría disponibles")
        return
    lat, lon = last_telemetry_pos
    set_home_position(lat, lon)
    log_message(f"🏠 HOME actualizado desde telemetría: {lat:.6f}, {lon:.6f}")
    global polygon_set
    polygon_set = True
    update_buttons_state()

def find_closest_polygon_point(lat, lon):
    if not current_mission.polygon_points:
        return None
    min_distance = inf
    closest_point = None
    for point in current_mission.polygon_points:
        dist = calculate_distance(lat, lon, point[0], point[1])
        if dist < min_distance:
            min_distance = dist
            closest_point = point
    return closest_point

def add_fire_marker(data):
    lat = data.get('lat')
    lon = data.get('lon')
    conf = data.get('confidence', None)

    if lat is None or lon is None:
        return

    # Construir etiqueta multilínea
    if conf is not None:
        label = f"FUEGO ({conf*100:.1f}%)\n{lat:.6f}, {lon:.6f}"
    else:
        label = f"FUEGO\n{lat:.6f}, {lon:.6f}"

    # Crear marcador principal
    marker = mapa.set_marker(
        lat, lon,
        text=label,
        marker_color_circle="orange",
        marker_color_outside="orange",
        text_color="black",
        font=("Arial", 9, "bold")
    )
    fire_markers.append(marker)

    # Log visual
    if conf is not None:
        log_message(f"🔥 Marcador FUEGO agregado: {lat:.6f}, {lon:.6f} (Confianza={conf*100:.1f}%)")
    else:
        log_message(f"🔥 Marcador FUEGO agregado: {lat:.6f}, {lon:.6f}")


def add_person_marker(data):
    lat = data.get('lat')
    lon = data.get('lon')
    conf = data.get('confidence', None)

    if lat is None or lon is None:
        return

    # Construir etiqueta multilínea
    if conf is not None:
        label = f"PERSONA ({conf*100:.1f}%)\n{lat:.6f}, {lon:.6f}"
    else:
        label = f"PERSONA\n{lat:.6f}, {lon:.6f}"

    # Crear marcador principal
    marker = mapa.set_marker(
        lat, lon,
        text=label,
        marker_color_circle="#8B4513",
        marker_color_outside="#8B4513",
        text_color="black",
        font=("Arial", 9, "bold")
    )
    person_markers.append(marker)

    # Log visual
    if conf is not None:
        log_message(f"🧍 Marcador PERSONA agregado: {lat:.6f}, {lon:.6f} (Confianza={conf*100:.1f}%)")
    else:
        log_message(f"🧍 Marcador PERSONA agregado: {lat:.6f}, {lon:.6f}")


def update_polygon_drawing():
    global polygon_path
    current_mission.polygon_points = order_polygon_points(current_mission.polygon_points)
    if polygon_path:
        safe_delete(polygon_path)
    if len(current_mission.polygon_points) >= 2:
        polygon_path = mapa.set_polygon(
            current_mission.polygon_points,
            outline_color="blue", fill_color="lightblue", border_width=2
 
         )
    refresh_mission_preview()

def order_polygon_points(points):
    """
    Ordena automáticamente los puntos del polígono según ángulo polar
    respecto al centroide, evitando cruces tipo 'moño'.
    """
    if len(points) < 3:
        return points

    cx = sum(p[0] for p in points) / len(points)
    cy = sum(p[1] for p in points) / len(points)

    ordered = sorted(points, key=lambda p: math.atan2(p[1] - cy, p[0] - cx))
    return ordered

def draw_mission_path(points):
    """
    Dibuja el path de la misión generada por la GS:
    - Path amarillo
    - Marcadores amarillos en cada WP
    - Log detallado con todos los puntos
    """
    global mission_path, mission_wp_markers

    # ----------------------------------------------------
    # BORRAR PATH ANTERIOR
    # ----------------------------------------------------
    if mission_path:
        try:
            safe_delete(mission_path)
        except:
            pass
        mission_path = None

    # ----------------------------------------------------
    # BORRAR MARCADORES ANTERIORES
    # ----------------------------------------------------
    for m in mission_wp_markers:
        try:
            safe_delete(m)
        except:
            pass
    mission_wp_markers.clear()

    # ----------------------------------------------------
    # VALIDACIÓN
    # ----------------------------------------------------
    if len(points) < 2:
        log_message("⚠️ draw_mission_path: muy pocos puntos")
        return

    log_message(f"🟨 Trazado GS: {len(points)} waypoints")

    # ----------------------------------------------------
    # LOGUEAR PUNTOS
    # ----------------------------------------------------
    for i, (lat, lon) in enumerate(points):
        log_message(f"   WP{i}: {lat:.7f}, {lon:.7f}")

    # ----------------------------------------------------
    # PATH AMARILLO
    # ----------------------------------------------------
    try:
        mission_path = mapa.set_path(points, color="yellow", width=3)
        log_message("🟨 Path GS dibujado")
    except Exception as e:
        log_message(f"⚠️ Error dibujando path GS: {e}")
        return

    # ----------------------------------------------------
    # MARCADORES AMARILLOS
    # ----------------------------------------------------
    for i, (lat, lon) in enumerate(points):
        try:
            marker = mapa.set_marker(lat, lon, text=f"GS{i}", marker_color_circle="yellow")
            mission_wp_markers.append(marker)
        except Exception as e:
            log_message(f"⚠️ Error creando marcador {i}: {e}")


def refresh_mission_preview():
    """Regenera el lawnmower automáticamente si hay polígono y HOME."""
    global mission_path

    if len(current_mission.polygon_points) < 3:
        if mission_path:
            safe_delete(mission_path)
            mission_path = None
        return

    # Actualizar parámetros de misión actuales
    current_mission.spacing = spacing_var.get()
    current_mission.detect_spacing = detect_spacing_var.get()

    # Generar path
    path = generate_lawnmower_path(
        current_mission.polygon_points,
        current_mission.spacing,
        current_mission.detect_spacing,
        home=current_mission.home_position
    )



    # Dibujarlo
    draw_mission_path(path)



def clear_all_waypoints():
    global waypoint_markers, active_wp_marker

    # Borrar marcador activo si existe
    if active_wp_marker:
        safe_delete(active_wp_marker)
        active_wp_marker = None

    # Borrar todos los demás waypoints
    for m in waypoint_markers:
        try:
            safe_delete(m)
        except:
            pass

    waypoint_markers.clear()
    print("🗑 Todos los waypoints eliminados")


def reset_polygon():
    global current_mission, polygon_markers, polygon_path
    global home_marker, home_line, mission_path
    global waypoint_status, person_markers, fire_markers
    global flight_track, flight_path
    global battery_fail_marker, battery_low_alert_shown   # 🟥 IMPORTANTE
    global mission_wp_markers, mission_path

    # Borrar marcadores del polígono
    for m in polygon_markers:
        safe_delete(m)
    polygon_markers.clear()

    # Borrar polígono dibujado
    if polygon_path is not None:
        try: 
            safe_delete(polygon_path)
        except Exception as e:
            log_message(f"Error borrando polígono: {e}")
        polygon_path = None

        # =====================================================================
    # 🟦 BORRAR TRAZADO DEL DRON (ESP)
    # =====================================================================
    global esp_mission_path, esp_wp_markers, esp_wp_chunks, esp_wp_total

    # Borrar línea celeste dibujada desde ESP
    if esp_mission_path:
        try:
            safe_delete(esp_mission_path)
        except:
            pass
        esp_mission_path = None

    # Borrar marcadores de ESP
    for m in esp_wp_markers:
        try:
            safe_delete(m)
        except:
            pass
    esp_wp_markers.clear()

    # Reset buffers de reconstrucción de fragmentos
    esp_wp_chunks = []
    esp_wp_total = 0

    log_message("🗑 Reset: Trazado del dron eliminado")


    # Limpiar ruta recorrida
    flight_track = []
    if flight_path:
        try:
            safe_delete(flight_path)
        except:
            pass
    flight_path = None

    # Borrar HOME
    if home_marker: safe_delete(home_marker)
    if home_line: safe_delete(home_line)
    home_marker = None
    home_line = None

    # Borrar marcadores de eventos PERSON
    for m in person_markers: 
        safe_delete(m)
    person_markers.clear()

    # Borrar marcadores de eventos FIRE
    for m in fire_markers: 
        safe_delete(m)
    fire_markers.clear()

    # BORRAR MARCADORES 
    for m in mission_wp_markers:
        try: safe_delete(m)
        except: pass
    mission_wp_markers.clear()

    # 🟨 BORRAR CAMINO DE MISIÓN (mission_path)
    if mission_path:
        try:
            safe_delete(mission_path)
        except:
            pass
    mission_path = None

    # 🟥 BORRAR MARCADOR DE BATERÍA BAJA
    if battery_fail_marker:
        try:
            safe_delete(battery_fail_marker)
        except:
            pass
        battery_fail_marker = None

    # 🟥 Permitir que futuras alertas de batería baja vuelvan a mostrarse
    battery_low_alert_shown = False

    # Reset de la misión
    current_mission = MissionData()
    waypoint_status = []

    # Reset UI elements
    home_position_var.set("")
    progress_var.set("Waypoints: 0/0")
    progress_bar['value'] = 0
    altitude_var.set(3)
    spacing_var.set(10)
    detect_spacing_var.set(10)

    clear_all_waypoints()

    log_message("🔄 Reset completo: polígono, HOME, misión, mission path y marcador de batería baja eliminados")

    polygon_set = False
    mission_sent = False
    update_buttons_state()
# =========================================
# [11] ARCHIVOS DE MISIÓN (GUARDAR/CARGAR)
# =========================================
def save_mission():
    if not current_mission.polygon_points:
        messagebox.showwarning("Error", "No hay misión para guardar")
        return
    filename = filedialog.asksaveasfilename(
        defaultextension=".mission",
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    if filename:
        try:
            with open(filename, 'wb') as f:
                pickle.dump(current_mission.to_dict(), f)
            log_message(f"Misión guardada: {filename}")
        except Exception as e:
            log_message(f"Error guardando misión: {str(e)}")

def load_mission():
    filename = filedialog.askopenfilename(
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    if filename:
        try:
            with open(filename, 'rb') as f:
                mission_data = pickle.load(f)
            reset_polygon()
            current_mission.from_dict(mission_data)
            update_interface_from_mission()
            log_message(f"Misión cargada: {filename}")
        except Exception as e:
            log_message(f"Error cargando misión: {str(e)}")

def update_interface_from_mission():
    for i, (lat, lon) in enumerate(current_mission.polygon_points):
        marker = mapa.set_marker(lat, lon, text=f"P{i+1}")
        polygon_markers.append(marker)
    if current_mission.home_position:
        set_home_position(current_mission.home_position[0], current_mission.home_position[1])
    altitude_var.set(current_mission.altitude)
    spacing_var.set(current_mission.spacing)
    detect_spacing_var.set(current_mission.detect_spacing)
    update_polygon_drawing()

# =========================================
# [12] CONSTRUCCIÓN DE LA GUI (TKINTER)
# =========================================
def create_trim_pid_controls(frame):
    """Panel reducido: SOLO Gripper y Simulación"""

    # === CONTENEDOR PRINCIPAL ===
    container = tk.Frame(frame)
    container.pack(fill="both", expand=True, padx=10, pady=10)
    container.grid_columnconfigure(0, weight=1)

    # =====================================================================
    # GRIPPER — NUEVA VERSIÓN CON DOS BOTONES
    # =====================================================================
    gripper_frame = tk.LabelFrame(container, text="Gripper", padx=10, pady=10, font=("Arial", 10, "bold"))
    gripper_frame.grid(row=0, column=0, sticky="ew", pady=(0, 8))
    
    def gripper_open():
        send_command("GRIPPER", {"state": "OPEN"})
        log_message("🟢 Gripper ABIERTO")
        gripper_open_btn.config(bg="#4CAF50")
        gripper_close_btn.config(bg="#f44336")
    
    def gripper_close():
        send_command("GRIPPER", {"state": "CLOSE"})
        log_message("🔴 Gripper CERRADO")
        gripper_close_btn.config(bg="#4CAF50")
        gripper_open_btn.config(bg="#f44336")
    
    # Botón ABRIR
    gripper_open_btn = tk.Button(
        gripper_frame, text="ABIERTO",
        font=("Arial", 11, "bold"),
        width=15,
        bg="#4CAF50", fg="white",
        command=gripper_open
    )
    gripper_open_btn.pack(pady=5)
    
    # Botón CERRAR
    gripper_close_btn = tk.Button(
        gripper_frame, text="CERRADO",
        font=("Arial", 11, "bold"),
        width=15,
        bg="#f44336", fg="white",
        command=gripper_close
    )
    gripper_close_btn.pack(pady=5)
    
    # Estado inicial
    # El gripper comienza en CERRADO
    gripper_close_btn.config(bg="#4CAF50")   # botón cerrado resaltado

    # =====================================================================
    # SIMULACIÓN
    # =====================================================================
    sim_frame = tk.LabelFrame(container, text="Simulación", padx=10, pady=10, font=("Arial", 10, "bold"))
    sim_frame.grid(row=1, column=0, sticky="ew", pady=(0, 8))

    global sim_active
    sim_active = tk.BooleanVar(value=False)

    def toggle_sim():
        state = sim_active.get()
        cmd = "SIM_ON" if state else "SIM_OFF"
        send_command(cmd, {})
        log_message("🧪 Simulación ACTIVADA" if state else "🛑 Simulación DESACTIVADA")
        sim_btn.config(bg="#4CAF50" if state else "#f44336", text="SIM ON" if state else "SIM OFF")

    sim_btn = tk.Checkbutton(sim_frame, text="SIM OFF", variable=sim_active, command=toggle_sim,
                             font=("Arial", 11, "bold"), indicatoron=False, width=15, bg="#f44336",
                             fg="white", selectcolor="#4CAF50")
    sim_btn.pack(pady=10)

    # =====================================================================
    # SOLICITAR TRAZADO DEL DRON (DEBUG WAYPOINTS)
    # =====================================================================
    req_frame = tk.LabelFrame(container, text="Trazado del Dron", padx=10, pady=10,font=("Arial", 10, "bold"))
    req_frame.grid(row=2, column=0, sticky="ew", pady=(0, 8))

    def request_esp_traj():
        send_command("REQ_WP_DEBUG", {})
        log_message("📡 Solicitud enviada: Trazado del dron (waypoints ESP)")

    req_btn = tk.Button(
        req_frame,
        text="📡 Solicitar Trazado",
        font=("Arial", 11, "bold"),
        width=15,
        bg="#2196F3",         # azul estilo GS
        fg="white",
        command=request_esp_traj
    )
    req_btn.pack(pady=10)

        # Botón para BORRAR trazado del dron (ESP)
    def clear_esp_traj():
        global esp_mission_path, esp_wp_markers, esp_wp_chunks, esp_wp_total

        # Borrar línea trazada
        if esp_mission_path is not None:
            try:
                safe_delete(esp_mission_path)
            except:
                pass
            esp_mission_path = None

        # Borrar marcadores
        for m in esp_wp_markers:
            try:
                safe_delete(m)
            except:
                pass
        esp_wp_markers.clear()

        # Reset buffers de reconstrucción
        esp_wp_chunks = []
        esp_wp_total = 0

        log_message("🗑 Trazado del dron eliminado")

    clear_btn = tk.Button(req_frame,text="🗑 Borrar Trazado",font=("Arial", 11, "bold"),width=15,bg="#f44336",fg="white",command=clear_esp_traj)
    clear_btn.pack(pady=(0,10))


def create_gui():
    """Construye toda la interfaz (pestañas, botones, mapa, etc.)."""
    global root, mapa, status_label, telemetry_text, log_text
    global port_var, spacing_var, altitude_var, status_message, detect_spacing_var
    global home_position_var, progress_var, progress_bar
    global add_btn, home_btn, delete_btn
    global btn_arm, btn_disarm, btn_mission, btn_return
    global event_action_var

    root = tk.Tk()
    root.title("Ground Station - Dron Buscador Térmico")
    root.geometry("1200x900")

      
    event_action_var = tk.StringVar(value="VOLVER")

    # ---- Barra de conexión ----
    connection_frame = tk.Frame(root)
    connection_frame.pack(fill="x", padx=10, pady=5)

    ports = [port.device for port in serial.tools.list_ports.comports()]
    port_var = tk.StringVar(value=ports[0] if ports else "")

    tk.Label(connection_frame, text="Puerto:").pack(side="left")
    port_combo = ttk.Combobox(connection_frame, textvariable=port_var, values=ports, width=15)
    port_combo.pack(side="left", padx=5)

    def update_ports():
        new_ports = [port.device for port in serial.tools.list_ports.comports()]
        port_combo['values'] = new_ports
        if new_ports and not port_var.get():
            port_var.set(new_ports[0])

    tk.Button(connection_frame, text="Actualizar", command=update_ports).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Conectar", command=connect_serial).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Desconectar", command=disconnect_serial).pack(side="left", padx=5)

    status_label = tk.Label(connection_frame, text="Desconectado", fg="red")
    status_label.pack(side="right")

    # ---- Modo edición ----
    edit_frame = tk.Frame(root)
    edit_frame.pack(fill="x", padx=10, pady=5)

    tk.Label(edit_frame, text="Modo Edición:", font=("Arial", 10, "bold")).pack(side="left")
    add_btn = tk.Button(edit_frame, text="Colocar marcadores", command=lambda: set_edit_mode('polygon'), width=20)
    add_btn.pack(side="left", padx=2)
    home_btn = tk.Button(edit_frame, text="Colocar home", command=set_home_from_telemetry, width=20)
    home_btn.pack(side="left", padx=2)
    delete_btn = tk.Button(edit_frame, text="Eliminar marcadores", command=lambda: set_edit_mode('delete'), width=20)
    delete_btn.pack(side="left", padx=2)

    # ---- Parámetros de misión ----
    params_frame = tk.Frame(root)
    params_frame.pack(fill="x", padx=10, pady=5)

    tk.Label(params_frame, text="Altitud (m):").pack(side="left")
    altitude_var = tk.IntVar(value=3)
    tk.Spinbox(params_frame, from_=1, to=100, width=5, textvariable=altitude_var).pack(side="left", padx=5)

    tk.Label(params_frame, text="Ancho pasadas (m):").pack(side="left", padx=(20, 5))
    spacing_var = tk.IntVar(value=10)
    tk.Spinbox(params_frame, from_=1, to=100, width=5, textvariable=spacing_var).pack(side="left", padx=5)
    def on_spacing_change(*args):
        refresh_mission_preview()    
    spacing_var.trace_add("write", on_spacing_change)

    tk.Label(params_frame, text="Espaciado detecciones (m):").pack(side="left", padx=(20, 5))
    global detect_spacing_var
    detect_spacing_var = tk.IntVar(value=10)
    tk.Spinbox(params_frame, from_=1, to=100, width=5, textvariable=detect_spacing_var).pack(side="left", padx=5)
    def on_detect_change(*args):
        refresh_mission_preview()
    detect_spacing_var.trace_add("write", on_detect_change)

    
    home_position_var = tk.StringVar(value="")
    tk.Label(params_frame, textvariable=home_position_var, fg="green").pack(side="right")

    # ---- Estado y progreso ----
    status_frame = tk.Frame(root)
    status_frame.pack(fill="x", padx=10, pady=5)

    status_message = tk.StringVar(value="Modo: Polígono")
    tk.Label(status_frame, textvariable=status_message, fg="white", bg="#333", font=("Arial", 12, "bold"),padx=8, pady=3).pack(side="left")


    progress_var = tk.StringVar(value="Waypoints: 0/0")
    tk.Label(status_frame, textvariable=progress_var).pack(side="left", padx=20)

    progress_bar = ttk.Progressbar(status_frame, orient="horizontal", length=200, mode="determinate")
    progress_bar.pack(side="left", padx=5)

    # ---- Botones de misión ----
    mission_btn_frame = tk.Frame(root)
    mission_btn_frame.pack(fill="x", padx=10, pady=5)

    btn_mission = tk.Button(mission_btn_frame, text="COMENZAR MISIÓN", command=start_mission,
                            state="disabled", bg="lightgrey", fg="white")
    btn_mission.pack(side="left", expand=True, padx=5)

    btn_return = tk.Button(mission_btn_frame, text="REGRESAR", command=return_home,
                           state="normal", bg="orange", fg="black")
    btn_return.pack(side="left", expand=True, padx=5)

    tk.Button(mission_btn_frame, text="RESET COMPLETO", command=reset_polygon, bg="red", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="GUARDAR", command=save_mission, bg="blue", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="CARGAR", command=load_mission, bg="purple", fg="white").pack(side="left", expand=True, padx=5)

    tk.Label(mission_btn_frame, text="Acción tras evento:").pack(side="left", padx=5)
    event_action_menu = ttk.Combobox(mission_btn_frame, textvariable=event_action_var,
                                     values=["CONTINUAR", "VOLVER"], state="readonly", width=11)
    event_action_menu.pack(side="left", padx=5)
    event_action_menu.current(1)  # "VOLVER"

    # ---- Notebook ----
    notebook = ttk.Notebook(root)
    notebook.pack(fill="both", expand=True, padx=10, pady=5)

    # Pestaña: Misión
    frame_mission = ttk.Frame(notebook)
    notebook.add(frame_mission, text="Misión")

    # Mapa
    global mapa
    mapa = tkintermapview.TkinterMapView(frame_mission, width=1000, height=500, corner_radius=0)
    mapa.pack(pady=10, fill="both", expand=True)
    mapa.set_position(-34.6037, -58.3816)
    mapa.set_zoom(12)
    mapa.add_left_click_map_command(map_click)
    mapa.set_tile_server(
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",max_zoom=19)

    # Pestaña: Controles
    frame_controls = ttk.Frame(notebook)
    notebook.add(frame_controls, text="Controles")

    control_btn_frame = tk.Frame(frame_controls)
    control_btn_frame.pack(pady=10)

    global btn_arm, btn_disarm, send_mission_btn
    
    send_mission_btn = tk.Button(control_btn_frame,text="ENVIAR MISIÓN",font=("Arial", 11, "bold"),width=15,bg="#03A9F4",fg="white",state="normal",
        command=lambda: prepare_and_send_mission())
    send_mission_btn.pack(pady=5)

    
    btn_arm = tk.Button(control_btn_frame, text="ARMAR", command=arm_drone, bg="green", fg="white", width=15)
    btn_arm.pack(side="left", padx=10)

    btn_disarm = tk.Button(control_btn_frame, text="DESARMAR", command=disarm_drone,
                           state="normal", bg="red", fg="white", width=15)
    btn_disarm.pack(side="left", padx=10)

    create_trim_pid_controls(frame_controls)


    # Pestaña: Telemetría
    frame_telemetry = ttk.Frame(notebook)
    notebook.add(frame_telemetry, text="Telemetría")

    global telemetry_text
    telemetry_text = scrolledtext.ScrolledText(frame_telemetry, width=80, height=15)
    telemetry_text.pack(pady=10, fill="both", expand=True)
    telemetry_text.insert("end", "Esperando datos de telemetría...\n")
    telemetry_text.config(state="disabled")

    # Pestaña: Log
    frame_log = ttk.Frame(notebook)
    notebook.add(frame_log, text="Log")

    global log_text
    log_text = scrolledtext.ScrolledText(frame_log, width=80, height=15)
    log_text.pack(pady=10, fill="both", expand=True)
    log_text.config(state="disabled")

    # Modo inicial
    set_edit_mode('polygon')

# =========================
# [13] INICIALIZACIÓN
# =========================
current_mission = MissionData()

# Ejecutar GUI 
create_gui()
log_message("✅ Interfaz lista. Conectá el puerto y espera telemetría...")
root.mainloop()



[SERIAL] ✅ Conectado a COM4 @ 115200 bps
[SERIAL] 🚀 Lanzando hilos: recepción y ack_manager
[RX ← LoRa/receive_data] Frame recibido ← UAV#{"t":"TELEMETRY","d":{"lat":-34.77427554,"lon":-58.45485658,"alt":1.718540342,"speed":1.034041464,"heading":152,"battery":87,"voltage":12.59446,"armed":false,"mode":"UNKNOWN","batt_warn":false},"ts":35475}#END
[RX ← LoRa/process_received_message] Mensaje recibido tipo 'TELEMETRY' → {'t': 'TELEMETRY', 'd': {'lat': -34.77427554, 'lon': -58.45485658, 'alt': 1.718540342, 'speed': 1.034041464, 'heading': 152, 'battery': 87, 'voltage': 12.59446, 'armed': False, 'mode': 'UNKNOWN', 'batt_warn': False}, 'ts': 35475}
[RX ← LoRa/receive_data] Frame recibido ← UAV#{"t":"TELEMETRY","d":{"lat":-34.77427798,"lon":-58.45485893,"alt":1.612079939,"speed":1.196517,"heading":152,"battery":87,"voltage":12.5883,"armed":false,"mode":"UNKNOWN","batt_warn":false},"ts":36812}#END
[RX ← LoRa/process_received_message] Mensaje recibido tipo 'TELEMETRY' → {'t': 'TELEMETRY', 'd': 